# Data Warehouse with Redshift - Create Tables
Use this notebook to develop the code before completing the `create_tables.py` file to create each of your tables.

>
> **Stephanie Anderton**  
> DEND Project \#3  
> May 29, 2019
>

In [1]:
import configparser
import psycopg2
from   sql_queries import create_table_queries, drop_table_queries
from   sql_queries import create_db_table_queries, drop_db_table_queries
import time
import re
import mylib
from   mylib import logger

## Set Flag for Loading Staging Tables
**NOTE:  Staging tables MUST exist before running this flag with False.**

In [2]:
# On first run through, this MUST be set to 1 
# unless you created the tables from the command line!

b_staging_tables = False

## Function Definitions

In [3]:
def drop_tables(cur, conn, table_queries):
    logger.info('Drop existing tables...')
    
    for query in table_queries:
        # the table name is the 5th word in the query string
        table = re.findall(r'\w+', query)[4]
        logger.info('delete table [ {} ]'.format(table))
        
        try:
            cur.execute(query)
            conn.commit()
        except psycopg2.Error as e: 
            logger.info('Error: Dropping table [ {} ]'.format(table))
            print(e)
            print(query)


In [4]:
def create_tables(cur, conn, table_queries):
    logger.info('Create tables...')
    
    for query in table_queries:
        # the table name is the 6th word in the query string
        table = re.findall(r'\w+', query)[5]
        logger.info('create table [ {} ]'.format(table))
        
        try:
            cur.execute(query)
            conn.commit()
        except psycopg2.Error as e: 
            logger.info('Error: Creating table [ {} ]'.format(table))
            print(e)
            print(query)


## Main()
### Setup DB connection

In [5]:
logger.info('---[ Create Tables ]---')
mylib.log_timestamp()
print("Logfile:  " + mylib.get_log_file_name())

# read config parameters for database connection string
config = configparser.ConfigParser()
config.read('dwh.cfg')

try:
    conn_string = "host={} dbname={} user={} password={} port={}"
    conn_string = conn_string.format(*config['CLUSTER'].values())
    conn = psycopg2.connect( conn_string )
    cur = conn.cursor()
    
    print(conn_string)
    logger.info('DB connection :  open')

except Exception as e:
    logger.info("Error: Could not make connection to the sparkify DB")
    print(e)

Logfile:  ./logs/etl-20190530.log
host=dwhcluster.cbsjbxldkge8.us-west-2.redshift.amazonaws.com dbname=sparkify user=dwhuser password=Passw0rd port=5439


### Drop & Create Tables

In [6]:
if b_staging_tables == True:
    drop_tables(cur, conn, drop_table_queries)
    create_tables(cur, conn, create_table_queries)
    
    # Now we can set flag to **False**, so on the next run through
    # I can then work on the dimensional tables only
    b_staging_tables = False

else:
    drop_tables(cur, conn, drop_db_table_queries)
    create_tables(cur, conn, create_db_table_queries)

In [7]:
# All done...
conn.close()
logger.info('DB connection :  closed')